<a href="https://colab.research.google.com/github/Sledger9/TCS-iON-RIO-210/blob/main/SemanticsErrorDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install happytransformer 
from IPython.display import clear_output
clear_output()

In [ ]:
import csv
from datasets import load_dataset
from happytransformer import TTSettings
from happytransformer import TTTrainArgs
from happytransformer import HappyTextToText

C:\Users\Shubhankar\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
happy_tt = HappyTextToText("T5", "t5-base")

C:\Users\Shubhankar\anaconda3\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
05/28/2023 17:55:18 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [ ]:
train_dataset = load_dataset("jfleg", split='validation[:]')

eval_dataset = load_dataset("jfleg", split='test[:]')

In [ ]:
for case in train_dataset["corrections"][:2]:
    print(case)
    print(case[0])
    print("--------------------------------------------------------")

['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
So I think we would not be alive if our ancestors did not develop sciences and technologies . 
--------------------------------------------------------
['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
Not for use with a car . 
--------------------------------------------------------


In [ ]:
def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
     	    # Adding the task's prefix to input 
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases contain blank strings. 
                if input_text and correction:
                    writter.writerow([input_text, correction])

In [ ]:
generate_csv("train.csv", train_dataset)
generate_csv("eval.csv", eval_dataset)

In [ ]:
pip install --upgrade accelerate

     -------------------------------------- 219.1/219.1 kB 4.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
before_result = happy_tt.eval("eval.csv")

05/28/2023 17:55:38 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/Shubhankar/.cache/huggingface/datasets/csv/default-95b972af66bdce9d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

C:\Users\Shubhankar\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
print("Before loss:", before_result.loss)

Before loss: 1.2803850173950195


In [ ]:
args = TTTrainArgs(batch_size=8)
happy_tt.train("train.csv", args=args)

05/28/2023 18:13:56 - INFO - happytransformer.happy_transformer -   Preprocessing training data...


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/Shubhankar/.cache/huggingface/datasets/csv/default-a30dd2e48ab17d8a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/3016 [00:00<?, ? examples/s]

05/28/2023 18:14:01 - INFO - happytransformer.happy_transformer -   Training...
C:\Users\Shubhankar\anaconda3\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.582400
1000,0.436500


In [ ]:
before_loss = happy_tt.eval("eval.csv")

print("After loss: ", before_loss.loss)

05/28/2023 19:35:57 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

After loss:  0.44646382331848145


In [ ]:
beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)

In [ ]:
""" Example1: """
example_1 = "grammar: This sentences, has bads grammar and spelling!"
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

This sentence has bad grammar and spelling!


In [ ]:
example_2 = "grammar: I am enjoys, writtings articles ons AI and I also enjoyed write articling on AI."

result_2 = happy_tt.generate_text(example_2, args=beam_settings)
print(result_2.text)

I enjoy writing articles on AI and I also enjoy writing articles on AI .
